# Object Detection Demo: Car Detection

This is a sample reference implementation to showcase object detection (car in this case) with single-shot detection (SSD) and Async API.
Async API improves the overall frame-rate of the application by not waiting for the inference to complete but continuing to do things on the host while inference accelerator is busy. 
Specifically, this code demonstrates two parallel inference requests by processing the current frame while the next input frame is being captured. This essentially hides the latency of frame capture.

## Overview of how it works
At start-up the sample application reads the equivalent of command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. 
A job is submitted to a hardware accelerator (Intel® Core CPU, Intel® HD Graphics GPU, Intel® Core CPU, Intel® Neural Compute Stick 2, Intel® Movidius™ Vision Processing Unit (8x VPU), and Intel® Arria® 10 FPGA.
After the inference is completed, the output videos are appropriately stored in the /results directory, which can then be viewed within the Jupyter Notebook instance. 

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes
* Demonstrate the Async API in action

## How do I modify the above demonstration objectives?
Modify the job_config.json file accompanying this example, this file is located in the same directory
It is possible to change the raw model used, change the precision of the IR files, or input video and target configurations as desired.


## Create an Intermediate Representation of the Model

Model Optimizer creates the Intermediate Representation of the model which is the device-agnostic, generic optimization of the model. Caffe\*, TensorFlow\*, MXNet\*, ONNX\*, and Kaldi\* models are supported by Model Optimizer.

We will use the **MobileNet-SSD** model for this example. Download the model, specifying the name and output directory.

In [1]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models

################|| Downloading models ||################

========== Downloading raw_models/public/mobilenet-ssd/mobilenet-ssd.prototxt
... 100%, 28 KB, 47796 KB/s, 0 seconds passed

========== Downloading raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel
... 100%, 22605 KB, 21878 KB/s, 1 seconds passed

################|| Post-processing ||################



Let's convert this model to the intermediate representation using Model Optimizer.

In [2]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--data_type FP32 \
--output_dir models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u19892/Downloads/i360Demo/iot-devcloud/python/demos/object-detection-python/raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/u19892/Downloads/i360Demo/iot-devcloud/python/demos/object-detection-python/models/mobilenet-ssd/FP32
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
Caffe specific parameters:
	- Path to Python Caffe* parser generated from caffe.proto: 	/opt/intel/openv

We will also need the FP16 IR version of the model for the calculations on Neural Compute Stick 2 (NCS2) and Visual Processing Unit (VPU); let's create it here.

In [3]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--data_type FP16 \
--output_dir models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u19892/Downloads/i360Demo/iot-devcloud/python/demos/object-detection-python/raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel
	- Path for generated IR: 	/home/u19892/Downloads/i360Demo/iot-devcloud/python/demos/object-detection-python/models/mobilenet-ssd/FP16
	- IR output name: 	mobilenet-ssd
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	[127,127,127]
	- Scale values: 	Not specified
	- Scale factor: 	256.0
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
Caffe specific parameters:
	- Path to Python Caffe* parser generated from caffe.proto: 	/opt/intel/openv

## Import libraries and configuration file for widget management.

In [4]:
import qarpo
import json
with open('job_config.json') as json_file:  
    data = json.load(json_file)

## Display available options for Intel® Edge compute hardware and acceleration,  
## Display results and performance summary.

In [5]:
job_interface = qarpo.Interface(data)
job_interface.displayUI()

Button(button_style='info', description='Submit', style=ButtonStyle())